# PromptLayer | プロンプトレイヤー> [PromptLayer](https://docs.promptlayer.com/introduction)は、プロンプトエンジニアリングのためのプラットフォームです。また、リクエストの可視化、プロンプトのバージョン管理、使用状況の追跡を通じて、LLMの可観測性を支援します。>> > [PromptLayer](https://docs.promptlayer.com/introduction) is a platform for prompt engineering. It also helps with the LLM observability to visualize requests, version prompts, and track usage.>> `PromptLayer`にはLangChainと直接統合するLLM（例えば[`PromptLayerOpenAI`](https://python.langchain.com/docs/integrations/llms/promptlayer_openai)）がありますが、`PromptLayer`をLangChainと統合する推奨される方法はコールバックを使用することです。>> > While `PromptLayer` does have LLMs that integrate directly with LangChain (e.g. [`PromptLayerOpenAI`](https://python.langchain.com/docs/integrations/llms/promptlayer_openai)), using a callback is the recommended way to integrate `PromptLayer` with LangChain.このガイドでは、`PromptLayerCallbackHandler`のセットアップ方法について説明します。> In this guide, we will go over how to setup the `PromptLayerCallbackHandler`.詳細については、[PromptLayerのドキュメント](https://docs.promptlayer.com/languages/langchain)をご覧ください。> See [PromptLayer docs](https://docs.promptlayer.com/languages/langchain) for more information.

## Installation and Setup | インストールとセットアップ

In [ ]:
!pip install promptlayer --upgrade

### Getting API Credentials | APIクレデンシャルの取得PromptLayerアカウントをお持ちでない場合は、[promptlayer.com](https://www.promptlayer.com)でアカウントを作成してください。その後、ナビゲーションバーの設定の歯車アイコンをクリックしてAPIキーを取得し、環境変数`PROMPTLAYER_API_KEY`として設定してください。> If you do not have a PromptLayer account, create one on [promptlayer.com](https://www.promptlayer.com). Then get an API key by clicking on the settings cog in the navbar and> set it as an environment variabled called `PROMPTLAYER_API_KEY`

## Usage | 使用方法`PromptLayerCallbackHandler`の使用を開始するのは非常に簡単で、2つのオプショナルな引数を取ります：> Getting started with `PromptLayerCallbackHandler` is fairly simple, it takes two optional arguments:1. `pl_tags` - PromptLayerで追跡されるタグとして機能する、オプションの文字列リストです。   > `pl_tags` - an optional list of strings that will be tracked as tags on PromptLayer.2. `pl_id_callback` - 引数として `promptlayer_request_id` を取る任意の関数です。このIDは、PromptLayerの追跡機能を使用してメタデータ、スコア、プロンプトの使用状況を追跡するために利用できます。   > `pl_id_callback` - an optional function that will take `promptlayer_request_id` as an argument. This ID can be used with all of PromptLayer's tracking features to track, metadata, scores, and prompt usage.

## Simple OpenAI Example | シンプルなOpenAIの例この簡単な例では、`ChatOpenAI` と `PromptLayerCallbackHandler` を使用します。`chatopenai` という名前の PromptLayer タグを追加します。> In this simple example we use `PromptLayerCallbackHandler` with `ChatOpenAI`. We add a PromptLayer tag named `chatopenai`

In [ ]:
import promptlayer  # Don't forget this 🍰
from langchain.callbacks import PromptLayerCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
)

chat_llm = ChatOpenAI(
    temperature=0,
    callbacks=[PromptLayerCallbackHandler(pl_tags=["chatopenai"])],
)
llm_results = chat_llm(
    [
        HumanMessage(content="What comes after 1,2,3 ?"),
        HumanMessage(content="Tell me another joke?"),
    ]
)
print(llm_results)

## GPT4All Example | GPT4Allの例

In [ ]:
import promptlayer  # Don't forget this 🍰
from langchain.callbacks import PromptLayerCallbackHandler
from langchain.llms import GPT4All

model = GPT4All(model="./models/gpt4all-model.bin", n_ctx=512, n_threads=8)

response = model(
    "Once upon a time, ",
    callbacks=[PromptLayerCallbackHandler(pl_tags=["langchain", "gpt4all"])],
)

## Full Featured Example | 多機能なサンプルこの例では、`PromptLayer`の機能をさらに引き出します。> In this example, we unlock more of the power of `PromptLayer`.PromptLayerを使用すると、プロンプトテンプレートを視覚的に作成、バージョン管理、追跡することができます。[Prompt Registry](https://docs.promptlayer.com/features/prompt-registry)を使用して、`example`という名前のプロンプトテンプレートをプログラムによって取得することが可能です。> PromptLayer allows you to visually create, version, and track prompt templates. Using the [Prompt Registry](https://docs.promptlayer.com/features/prompt-registry), we can programmatically fetch the prompt template called `example`.また、`promptlayer_request_id`を受け取り、スコア、メタデータを記録し、使用されたプロンプトテンプレートをリンクする`pl_id_callback`関数を定義しています。トラッキングについての詳細は、[当社のドキュメント](https://docs.promptlayer.com/features/prompt-history/request-id)でご覧ください。> We also define a `pl_id_callback` function which takes in the `promptlayer_request_id` and logs a score, metadata and links the prompt template used. Read more about tracking on [our docs](https://docs.promptlayer.com/features/prompt-history/request-id).

In [ ]:
import promptlayer  # Don't forget this 🍰
from langchain.callbacks import PromptLayerCallbackHandler
from langchain.llms import OpenAI


def pl_id_callback(promptlayer_request_id):
    print("prompt layer id ", promptlayer_request_id)
    promptlayer.track.score(
        request_id=promptlayer_request_id, score=100
    )  # score is an integer 0-100
    promptlayer.track.metadata(
        request_id=promptlayer_request_id, metadata={"foo": "bar"}
    )  # metadata is a dictionary of key value pairs that is tracked on PromptLayer
    promptlayer.track.prompt(
        request_id=promptlayer_request_id,
        prompt_name="example",
        prompt_input_variables={"product": "toasters"},
        version=1,
    )  # link the request to a prompt template


openai_llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    callbacks=[PromptLayerCallbackHandler(pl_id_callback=pl_id_callback)],
)

example_prompt = promptlayer.prompts.get("example", version=1, langchain=True)
openai_llm(example_prompt.format(product="toasters"))

それだけで完了です！セットアップ後は、あなたのすべてのリクエストがPromptLayerダッシュボードに表示されます。このコールバックは、LangChainに実装されたどのLLMにも対応しています。> That is all it takes! After setup all your requests will show up on the PromptLayer dashboard.> This callback also works with any LLM implemented on LangChain.